In [1]:
# Setup

from __future__ import print_function

import os
import subprocess

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
# MAP - Mean Average Precision
# code from https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [7]:
# Nearest Neighbor Classifier

from sklearn.neighbors import KNeighborsClassifier

# Load in the testing and training datasets
trainFull = pd.read_csv("data/booking_no_empty.csv", nrows=1000000)
train = trainFull[:800000].sample(100000)
test_set = trainFull[800000:].sample(100000)

# test = pd.read_csv("data/train.csv", nrows = 1000000)
# test_set = test.sample(100000)

# Take subsets of the datasets for training and testing
# train = trainFull[:800000].sample(100000)
# test_set = trainFull[800000:].sample(100000)

# Set a list of features to be considered in the tree
features = trainFull.columns.values.tolist()
removelist  =  ["hotel_cluster", "user_id", "date_time", 
"orig_destination_distance", "srch_ci", "srch_co"]
features = [column for column in features if column not in removelist]
print("The features considered are:")
print(features)

# Create and fit a decision tree to the set of data in those features
y = trainFull["hotel_cluster"] 
X = trainFull[features]
kn = KNeighborsClassifier(n_neighbors=25, weights='distance', algorithm='kd_tree', n_jobs=-1)
kn.fit(X, y)

# Score ability to predict the right hotel clust for a new subset
testX = test_set[features]
testy = test_set["hotel_cluster"]
score = kn.score(testX, testy)

# map5 = mapk(testX, testy, 5)

print("Score is " + str(score))
# print("MAP@5 score is " + str(map5))

The features considered are:
['site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market']
Score is 0.89722


In [ ]:
# Random Forest 

from sklearn.ensemble import RandomForestClassifier

# Load in the testing and training datasets
trainFull = pd.read_csv("data/train.csv", nrows=1000000)

# Take subsets of the datasets for training and testing
train = trainFull[:800000].sample(100000)
test_set = trainFull[800000:].sample(100000)

# Set a list of features to be considered in the tree
features = trainFull.columns.values.tolist()
removelist  =  ["hotel_cluster", "user_id", "date_time", 
"orig_destination_distance", "srch_ci", "srch_co"]
features = [column for column in features if column not in removelist]
print("The features considered are:")
print(features)

# Create and fit a decision tree to the set of data in those features
y = trainFull["hotel_cluster"] 
X = trainFull[features]
rf = RandomForestClassifier(n_estimators=20, n_jobs=-1, max_features=None)
rf.fit(X, y)

# Print the Unique Classes
print("Unique Classes: {}".format(rf.classes_))

# Print feature importances
print(rf.feature_importances_)

# Score ability to predict the right hotel clust for a new subset
testX = test_set[features]
testy = test_set["hotel_cluster"]
score = rf.score(testX, testy)

print("Score is " + str(score))

In [ ]:
# One vs. Rest

from __future__ import print_function

import os
import subprocess
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Load in the testing and training datasets
trainFull = pd.read_csv("data/train.csv", nrows=100000)

# Take subsets of the datasets for training and testing
train = trainFull[:80000].sample(10000)
test_set = trainFull[80000:].sample(10000)

# Set a list of features to be considered in the tree
features = trainFull.columns.values.tolist()
removelist  =  ["hotel_cluster", "user_id", "date_time", 
"orig_destination_distance", "srch_ci", "srch_co"]
features = [column for column in features if column not in removelist]
print("The features considered are:")
print(features)

# Create and fit a decision tree to the set of data in those features
y = trainFull["hotel_cluster"] 
X = trainFull[features]
ovr = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=25, weights='distance', algorithm='kd_tree', n_jobs=-1), n_jobs=-1)
ovr.fit(X, y)

# Score ability to predict the right hotel clust for a new subset
testX = test_set[features]
testy = test_set["hotel_cluster"]
score = ovr.score(testX, testy)

print("Score is " + str(score))

In [ ]:
# Support Vector Machine

from __future__ import print_function

import os
import subprocess
from sklearn.svm import SVC

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Load in the testing and training datasets
trainFull = pd.read_csv("data/train.csv", nrows=100000)

# Take subsets of the datasets for training and testing
train = trainFull[:80000].sample(10000)
test_set = trainFull[80000:].sample(10000)

# Set a list of features to be considered in the tree
features = trainFull.columns.values.tolist()
removelist  =  ["hotel_cluster", "user_id", "date_time", 
"orig_destination_distance", "srch_ci", "srch_co"]
features = [column for column in features if column not in removelist]
print("The features considered are:")
print(features)

# Create and fit a decision tree to the set of data in those features
y = trainFull["hotel_cluster"] 
X = trainFull[features]
svc = SVC(kernel='poly')
svc.fit(X, y)

# Score ability to predict the right hotel clust for a new subset
testX = test_set[features]
testy = test_set["hotel_cluster"]
score = svc.score(testX, testy)

print("Score is " + str(score))

The features considered are:
['site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market']
